In [1]:
import os
import sys
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor

In [3]:
# code for hpc:
# labels_path = '/groups/CS156b/data/student_labels/train2023.csv'
# img_dir = '/groups/CS156b/data/train'

# code for local:
labels_path = 'data/train/labels/labels.csv'
img_dir = 'data/train/images'

df = pd.read_csv(labels_path, delimiter='\t')[:-1]
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Pneumonia,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,0,0,0,train/pid50512/study1/view1_frontal.jpg,Female,68,Frontal,AP,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,1,1,1,train/pid21580/study2/view1_frontal.jpg,Female,87,Frontal,AP,-1.0,NaN,0.0,1.0,NaN,0.0,NaN,1.0,NaN
2,2,2,2,train/pid21580/study1/view1_frontal.jpg,Female,83,Frontal,AP,-1.0,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN
3,3,3,3,train/pid21580/study1/view2_lateral.jpg,Female,83,Lateral,NaN,-1.0,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN
4,4,4,4,train/pid33839/study1/view1_frontal.jpg,Male,41,Frontal,AP,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file, delimiter='\t')[:-1]
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        row = self.img_labels.iloc[idx]

        img_path = row['Path'].split('/')
        img_path = '/'.join(img_path[1:])
        img_path = os.path.join(self.img_dir, img_path)

        image = read_image(img_path)
        label = list(row[-9:]) # extract label, the last 9 columns

        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)

        return image, label

In [36]:
training_data = CustomImageDataset(labels_path, img_dir)
a, b = training_data[0]
a, b

(tensor([[[  1,   1,   1,  ...,   0,   0,   0],
          [  0,   0,   0,  ...,   0,   0,   0],
          [  1,   1,   1,  ...,   1,   1,   1],
          ...,
          [172, 158, 152,  ..., 233, 200, 242],
          [163, 160, 137,  ..., 239, 231, 255],
          [175, 162, 130,  ..., 207, 205, 255]]], dtype=torch.uint8),
 [1.0, nan, nan, nan, nan, nan, nan, nan, 1.0])